# Data wrangling

## Handle missing values

### Identify missing data

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from pickle import Pickler, Unpickler
import numpy as np
import pickle

df = pd.read_csv('../data/raw/yield_df.csv', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28242 entries, 0 to 28241
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Area                           28242 non-null  object 
 1   Item                           28242 non-null  object 
 2   Year                           28242 non-null  int64  
 3   hg/ha_yield                    28242 non-null  int64  
 4   average_rain_fall_mm_per_year  28242 non-null  float64
 5   pesticides_tonnes              28242 non-null  float64
 6   avg_temp                       28242 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.7+ MB


In [2]:
df.isna().sum()

Area                             0
Item                             0
Year                             0
hg/ha_yield                      0
average_rain_fall_mm_per_year    0
pesticides_tonnes                0
avg_temp                         0
dtype: int64

+ In the dataset there are no missing values
+ No correcting action is required.

---
## Outlier identifier

### Identify outliers

In [3]:
def find_outliers(x):
    q1 = np.quantile(x, 0.25, axis=0)
    q3 = np.quantile(x, 0.75, axis=0)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return (x >= lower) & (x <= upper)

In [4]:
# Removing outliers from hg/ha_yield
df = df[np.all(find_outliers(
    df[['hg/ha_yield', 'avg_temp', 'average_rain_fall_mm_per_year']]), axis=1)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26152 entries, 0 to 28241
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Area                           26152 non-null  object 
 1   Item                           26152 non-null  object 
 2   Year                           26152 non-null  int64  
 3   hg/ha_yield                    26152 non-null  int64  
 4   average_rain_fall_mm_per_year  26152 non-null  float64
 5   pesticides_tonnes              26152 non-null  float64
 6   avg_temp                       26152 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.6+ MB


- **Identify outliers**: there are about 2000 outliers present in columns `hg/ha_yield`, `avg_temp` and `average_rain_fall_mm_per_year` (mostly in the yield field).
- **Handle outliers**: outliers can be dropped from the dataset for a  more precise approach.

---
## Feature engineering and encoding

### Feature reduction


In [5]:
df.drop(['pesticides_tonnes'], axis=1, inplace=True)
df.to_csv('../data/processed/yield.csv', index=False)
df.head()

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,avg_temp
0,Albania,Maize,1990,36613,1485.0,16.37
1,Albania,Potatoes,1990,66667,1485.0,16.37
2,Albania,Rice,1990,23333,1485.0,16.37
3,Albania,Sorghum,1990,12500,1485.0,16.37
4,Albania,Soybeans,1990,7000,1485.0,16.37



### Encoding categorical variables

In [6]:
encoder = OneHotEncoder(sparse_output=False).fit(
    df[['Item', 'Area']])
encoded_values = pd.DataFrame(encoder.transform(
    df[['Item', 'Area']]), columns=encoder.get_feature_names_out())
df = df.drop(['Area', 'Item'], axis=1).join(encoded_values)
df.head()

,Year,hg/ha_yield,average_rain_fall_mm_per_year,avg_temp,Item_Cassava,Item_Maize,Item_Plantains,Item_Potatoes,Item_Rice,Item_Sorghum,...,Area_Tajikistan,Area_Thailand,Area_Tunisia,Area_Turkey,Area_Uganda,Area_Ukraine,Area_United Kingdom,Area_Uruguay,Area_Zambia,Area_Zimbabwe
0,1990,36613,1485.0,16.37,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1990,66667,1485.0,16.37,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1990,23333,1485.0,16.37,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1990,12500,1485.0,16.37,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1990,7000,1485.0,16.37,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Save encoder object

In [7]:
encoder_file = open('../artifacts/encoder.pkl', 'wb')
pickler = Pickler(encoder_file)
pickler.dump(encoder)
encoder_file.close()
encoder_file = open('../artifacts/encoder.pkl', 'rb')
unpickler = Unpickler(encoder_file)
unpickler.load()

OneHotEncoder(sparse_output=False)

- **Feature reduction**: `pesticides_tonnes` (uninteresting for the application) are dropped from the dataframe. The updated dataframe is saved to `data/processed/yield.csv`.
- **Feature transformation**: information encoded in each column is atomic so no splitting is required. Also, no merge is necessary.
- **Encoding categorical variables**: variables in columns `Area` and `Item` are encoded with `sklearn.preprocessing.OneHotEncoder`.
- **Save encoder object**: pickled scaler object to `artifacts/encoder.pkl`

---
## Data splitting

In [8]:
X = df.drop(['hg/ha_yield'], axis=1)
X.head()

,Year,average_rain_fall_mm_per_year,avg_temp,Item_Cassava,Item_Maize,Item_Plantains,Item_Potatoes,Item_Rice,Item_Sorghum,Item_Soybeans,...,Area_Tajikistan,Area_Thailand,Area_Tunisia,Area_Turkey,Area_Uganda,Area_Ukraine,Area_United Kingdom,Area_Uruguay,Area_Zambia,Area_Zimbabwe
0,1990,1485.0,16.37,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1990,1485.0,16.37,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1990,1485.0,16.37,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1990,1485.0,16.37,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1990,1485.0,16.37,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
y = df['hg/ha_yield']
y.head()

0    36613
1    66667
2    23333
3    12500
4     7000
Name: hg/ha_yield, dtype: int64


---

## Scaling numerical features

### Standardization

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X)

### Save scaler object

In [11]:
with open('../artifacts/scaler.pkl', 'wb') as f:
    pickler = pickle.Pickler(f)
    pickler.dump(scaler)

with open('../artifacts/scaler.pkl', 'rb') as f:
    unpickler = pickle.Unpickler(f)
    scaler_loaded = unpickler.load()

StandardScaler()

+ **Normalization**: normalized data (now all numeric) with `StandardScaler`
+ **Save scaler object**: pickled scaler object to file `artifacst/scaler.pkl`

---

## Conclusions
Summarizing:
- The dataset did not present missing data.
- Outliers are dropped from the dataset.
- Column `pesticides_tonnes` is dropped as it doesn't encode relevant information. The cleaned dataset is saved to `data/processed/yield.csv`.
- Categorical variables are encoded with `sklearn.preprocessing.OneHotEncoder` which is saved to file `artifacts/encoder.pkl`.
- Data is scaled using `sklearn.preprocessing.StandardScaler` which is saved to the file `artifacts/scaler.pkl`.